## **Análisis de Sistemas Térmicos usando Python**

Clase 8:    Uso de librería fluids para evaluar flujo en tuberías - **fsolve** 

In [ ]:
#pip install CoolProp

In [ ]:
#pip install fluids

In [ ]:
#pip install pint

La figura muestra un esquema de un sistema de dos estanques de agua conectados por dos cañerías con una bifurcación. 

Buscamos determinar el caudal que fluye entre los estanques, cuando la válvula en la sección 2 está completamente cerrada.


![Ejemplo 3](M1_E3.png)

Datos:
* Cañerías de acero comercial
* Tramo 1: $L_1$ = 3 m, $D_1$ = 4"
* Tramo 2: $L_2$ = 90 m, $D_2$ = 3", 2 codos de 45º, 1 codo de 90º
* Tramo 3: $L_3$ = 150 m, , $D_3$ = 4", 1 codo de 90º
* Tramo 4: $L_4$ = 3 m, $D_4$ = 4"
* Uniones en T son despreciadas para el cálculo de pérdidas de carga.

Importamos las librerías que utilizaremos. Se destaca la función **fsolve** de scipy.optimize, que nos permitirá resolver sistemas de ecuaciones no lineales.

In [2]:
import CoolProp.CoolProp as cp
import fluids as fld
import numpy as np
from scipy.constants import g, inch
from fluids.units import *
from scipy.optimize import fsolve

Luego definimos las variables asociadas a los datos de entrada del problema. Aprovechamos de utilizar la extensión de unidades del paquete **fluids**.

In [3]:
D_1 = 4 * inch * u.m #Pulgadas a metros
D_2 = 3 * inch * u.m  #Pulgadas a metros
D_3 = 4 * inch * u.m  #Pulgadas a metros
D_4 = 4 * inch * u.m  #Pulgadas a metros

L_1 = 3 * u.m   #metros
L_2 = 90 * u.m  #metros
L_3 = 150 * u.m #metros
L_4 = 3 * u.m 

z = 1 * u.m     #metros

fluid = "water"
T_o = 300 *u.K #Kelvin
rho = cp.PropsSI('D','T', T_o.magnitude, 'P',101325,fluid)*u.kg/u.m**3
mu = cp.PropsSI('V','T', T_o.magnitude, 'P',101325,fluid)*u.Pa*u.s

Si la válvula en la cañería 2 está cerrada, no hay flujo en la sección 2

$V_2 = 0$ - Velocidad Nula

$\dot{V}_2 = 0$ - Caudal Nulo

La ecuación de Bernouilli se puede establecer para cada uno de los tramos de tubería y luego pueden sumarse entre sí. 

Haciendo Bernoulli para el flujo que pasa por los tramos 1, 3 y 4, queda:

 $\rho g z_{AB}=\left( {\frac{1}{2}\rho V_{1}^{2}\left( {\frac{f_1 L_1}{D_1}+K_1} \right)} \right)+\frac{1}{2}\rho V_{3}^{2}\left( {\frac{f_3 L_3}{D_3}+K_3} \right) + \left( {\frac{1}{2}\rho V_{4}^{2}\left( {\frac{f_4 L_4}{D_4}+K_4} \right)} \right)$

 Como no hay flujo en 2 y como $D_1=D_3=D_4$, entonces $V_1 =V_3 =V_4$.
 
 Luego,

  $\rho g \Delta z= \frac{1}{2}\rho V_{1}^{2}\left(f_1 \frac{(L_1 + L_3 + L_4) }{D_1}+ \sum_i{K_i} \right)$

Para aplicar entonces la ecuación de Bernoulli, debemos determinar los valores de los coeficientes de pérdidas menores, así como la rugosidad del material. 

Luego. Haciendo uso de los diccionarios de la librería **fluids**, tenemos:

In [4]:
K_elbow_90 = fld.fittings.bend_rounded_Crane(Di=D_1.magnitude, bend_diameters=1, angle=90)*u.dimensionless
K_exit = fld.exit_normal()*u.dimensionless
K_entrance = fld.entrance_sharp()*u.dimensionless

material = "Commercial steel"
epsilon = fld.material_roughness(material)*u.m

La diferencia de presión hidrostática entre los estánques induce el caudal que buscamos determinar.

Esta diferencia de presión se puede define como:

In [100]:
g = g * u.m / u.s**2
DP_g = rho * g * z

En este problema, como el caudal es desconocido, también lo es la velocidad del flujo. En dicho contexto, el factor de fricción también es desconocido.
Así, la solución pasa a ser un problema iterativo. 

Para resolver el problema se pueden utilizar tecnicas de optimización, como el método de newton, entre otros. En este caso, se utilizará la función de **scipy.optimize.fsolve** que busca raíces a partir de un valor inicial.

Definimos primero la función a resolver que consiera el flujo por las secciones 1-3-4

In [125]:
def flow_134(V_dot):
    V_dot = V_dot * u.m**3/u.s
    DP_f = fld.one_phase_dP(V_dot*rho, rho, mu, D_1 ,epsilon, L_1+L_3+L_4)
    DP_s =(K_elbow_90+K_exit+K_entrance)*((V_dot*4)/(np.pi*D_1**2))**2 * 0.5*rho
    return (DP_f.magnitude + DP_s.magnitude - DP_g.magnitude)

Cabe destacar que las funciones de la biblioteca optimize requieren que los argumentos sean escalares, por eso se usan solo las magnitudes de las variables.

Ahora, definimos un valor inicial de 10 lts/s (arbitrario) y resolvemos usando **fsolve**.

In [126]:
V_dot = (10*u.L/u.s).to(u.m**3/u.s)
V_dot_134 = (fsolve(flow_134,V_dot))*u.m**3/u.s
V_dot_134.to(u.L/u.s)

Magnitude,[4.523559498208209]
Units,liter/second


## Parte B

¿Qué sucede si la válvula se abre?

![Ejemplo 3](M1_E3.png)

En dicho caso el agua fluirá por ambas seciones: 2 y 3

Así, el problema agrega una complejidad adicional pues la pérdida de carga observada entre las secciones 2 y 3 deben ser equivamentes. O sea

$\Delta P_2 = \Delta P_3$

Asimismo, el flujo másico que fluye por la seccion 1 debe distribuirse entre las secciones 2 y 3. 

De esta forma (asumiendo que el flujo es incompresible):

$\dot{V}_1 = \dot{V}_2 + \dot{V}_3$

Para resolver, primero definimos los coeficientes de las singularidades de la sección 2, que no fueron definidos previamente.

In [109]:
K_elbow_45 = fld.fittings.bend_rounded_Crane(Di=D_2.magnitude, bend_diameters=1, angle=45)*u.dimensionless
K_elbow_90_2 = fld.fittings.bend_rounded_Crane(Di=D_2.magnitude, bend_diameters=1, angle=90)*u.dimensionless
K_globe = fld.fittings.K_gate_valve_Crane(D_2.magnitude, D_2.magnitude, 0)*u.dimensionless

Luego, a modo de facilitar la compreesión del código, definimos una función para determinar la pérdida de carga en cada una de las secciones o tramos de tubería. En estas funciones se determinan las pérdidas de carga asociadas a la fricción y a las singularidades.

In [136]:
def flow_1(V_dot):
    V_dot = V_dot * u.m**3/u.s
    DP_f_1 = fld.one_phase_dP(V_dot*rho, rho, mu, D_1 ,epsilon, L_1)
    DP_s_1 =(K_entrance)*((V_dot*4)/(np.pi*D_1**2))**2 * 0.5*rho
    return (DP_f_1 + DP_s_1).magnitude

In [137]:
def flow_2(V_dot):
    V_dot = V_dot * u.m**3/u.s
    DP_f_2 = fld.one_phase_dP(V_dot*rho, rho, mu, D_1 ,epsilon, L_2)
    DP_s_2 =(K_globe + 2* K_elbow_45 + K_elbow_90_2)*((V_dot*4)/(np.pi*D_2**2))**2 * 0.5*rho
    return (DP_f_2 + DP_s_2).magnitude

In [138]:
def flow_3(V_dot):
    V_dot = V_dot * u.m**3/u.s
    DP_f_3 = fld.one_phase_dP(V_dot*rho, rho, mu, D_3 ,epsilon, L_3)
    DP_s_3 =(K_elbow_90)*((V_dot*4)/(np.pi*D_3**2))**2 * 0.5 * rho
    return (DP_f_3 + DP_s_3).magnitude

In [139]:
def flow_4(V_dot):
    V_dot = V_dot * u.m**3/u.s
    DP_f_4 = fld.one_phase_dP(V_dot*rho, rho, mu, D_4 ,epsilon, L_4)
    DP_s_4 =(K_exit)*((V_dot*4)/(np.pi*D_4**2))**2 * 0.5*rho
    return (DP_f_4 + DP_s_4).magnitude

Ahora, definimos las ecuaciones no lineales que constituyen el sistema a resolver:

* Pérdida de carga equivalente entre las secciones 2 y 3 
* Conservación de energía (Bernoulli) para el flujo pasando por las secciones 1, 2 y 4 (Es equivalente a la conservación de energía entre las secciones 1, 3 y 4)

In [140]:
def Flow (x):
    return [flow_2(x[0]*x[1])-flow_3(x[0]*(1-x[1])) , flow_1(x[0])+flow_2(x[0]* x[1])+flow_4(x[0]) - DP_g.magnitude]

Luego, resolvemos el sistema de ecuaciones usando la función **fsolve** de la librería **scipy.optimize**. Esta función requiere como argumentos la función que contiene el sistema de ecuaciones y un vector con los valores iniciales de las variables. La función **fsolve** devuelve un vector con los valores de las variables que hacen que el sistema de ecuaciones sea igual a cero.

In [146]:
root = fsolve(Flow, [10/1000, 0.5])
V_dot_open = (root[0]*u.m**3/u.s).to(u.L/u.s)

print(root[1])
V_dot_open

0.5542493037485157


9.072240898293497 <Unit('liter / second')>